In [ ]:
def build_emc_workflow(fdata, fbvals, fbvecs, work_dir='/tmp'):
    wf_emc.base_dir = work_dir

    wf_emc = pe.Workflow(name='emc_correction')

    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_files', 'in_bval',
                                                      'in_bvec']),
        name='inputnode')

    inputnode.inputs.dwi_file = fdata
    inputnode.inputs.in_bval = fbvals
    inputnode.inputs.in_bvec = fbvecs

    gradient_table = pe.Node(CheckGradientTable(), name="gradient_table")

    dwi_reference_wf = init_dwi_reference_wf()

    emc_wf_node = init_emc_wf('emc_wf')

    wf_emc.connect([
        (inputnode, gradient_table, [("dwi_file", "dwi_file"),
                                 ("in_bvec", "in_bvec"),
                                 ("in_bval", "in_bval")]),
        (inputnode, dwi_reference_wf, [("dwi_file", "inputnode.dwi_file")]),
        (gradient_table, dwi_reference_wf, [("b0_ixs", "inputnode.b0_ixs")]),
        (dwi_reference_wf, emc_wf_node, [("outputnode.ref_image",
                                          'meta_inputnode.b0_template')]),

        (inputnode, emc_wf_node, [('dwi_file', 'meta_inputnode.dwi_file'),
                                  ('in_bval', 'meta_inputnode.in_bval'),
                                  ('in_bvec', 'meta_inputnode.in_bvec')])
    ])

    cfg = dict(execution={'stop_on_first_crash': False,
                          'parameterize_dirs': True, 'crashfile_format': 'txt',
                          'remove_unnecessary_outputs': False,
                          'poll_sleep_duration': 0, 'plugin': 'MultiProc'})
    for key in cfg.keys():
        for setting, value in cfg[key].items():
            wf_emc.config[key][setting] = value

    return wf_emc

wf = build_emc_workflow(fdata, fbvals, fbvecs)
res = wf.run()